# Imports

In [1]:
import pandas as pd

In [2]:
!ls

README.md  Untitled.ipynb


In [4]:
!git add --a

In [5]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   Testbook.ipynb



In [6]:
!git commit -m "Create Notebook"

[master 26f45b1] Create Notebook
 Committer: EC2 Default User <ec2-user@ip-172-16-60-195.us-east-2.compute.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 1 file changed, 65 insertions(+)
 create mode 100644 Testbook.ipynb
